In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv

_ : bool = load_dotenv(find_dotenv())

MAPBOX_TOKEN = os.environ.get("MAPBOX_TOKEN")


client:OpenAI = OpenAI()

In [2]:
# import pandas as pd
import plotly.graph_objects as go



suggestLocations:[dict] = [
                           {
    "name":"Marysville",
    'lat':48.0517637,
    'lon':-122.1770818,
},
    {
        "name":"Perris",
        'lat':33.7825194,
        'lon':-117.22864779999999,
    }
]

mainLocation:dict = suggestLocations[0]
# City,State,Population,lat,lon
# Marysville,Washington,63269,48.0517637,-122.1770818
# Perris,California,72326,33.7825194,-117.22864779999999
# Cleveland,Ohio,390113,41.499320000000004,-81.6943605
# Worcester,Massachusetts,182544,42.262593200000005,-71.8022934
# Columbia,South Carolina,133358,34.0007104,-81.0348144
# Waterbury,Connecticut,109676,41.558152500000006,-73.0514965
# Eagan,Minnesota,65453,44.804132200000005,-93.1668858
# Southfield,Michigan,73006,42.473368799999996,-83.2218731






def travel_map(mainLocation, suggestLocations):
    # mapData = pd.DataFrame(data=data)
    print(mainLocation)
    print(suggestLocations)
    lat = [sl['lat'] for sl in suggestLocations]
    lon = [sl['lon'] for sl in suggestLocations]
    name = [sl['name'] for sl in suggestLocations]

    fig = go.Figure(go.Scattermapbox(
        # df=mapData,
        mode = "markers",
        lon = lon, lat = lat,
        hovertext=name,
        marker=dict(
            color='red',
            size=10,
           
        ),
        textposition = "bottom right"))

    fig.update_layout(
        mapbox = {
            'zoom':15,
            'accesstoken': MAPBOX_TOKEN,
            'style': "outdoors",
            "center":{'lat': mainLocation['lat'], 'lon': mainLocation['lon']}
            
            },
        showlegend = False)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.update_traces(marker_color='red', selector=dict(type='scattermapbox'))
    fig.show()

# travel_map(mainLocation,suggestLocations)


In [3]:
# creating assistant

assistant = client.beta.assistants.create(
    name="Travel Assistant",
    instructions="""
        you are a experienced travel assiatant, your job is to suggest the user best places at that location which is provided by the user
        function retutns "done" which means uesr got the suggestion
    """,
    model="gpt-3.5-turbo-1106",
    tools=[
        {
            "type":'function',
            "function":{
                "name":"travel_map",
                "description":"""
                    
                    { "name", 'lat', 'lon' }
                    """,
               "parameters": {
                    "type": "object",
                    "properties": {
                    "mainLocation": {"type": "object", "description": "this is the main locaiton and also the center point of map"},
                    "suggestLocations": {"type": "object", "description": "this would be the list of suggest locaitons"},
                },
                "required": ["mainLocation"]
                }

            }
        }
    ]
)

In [4]:
from openai.types.beta.thread import Thread

thread: Thread  = client.beta.threads.create()

print(thread)

Thread(id='thread_fSq2Ia3jB5OF0Qb6pkW9XE7t', created_at=1706980254, metadata={}, object='thread')


In [5]:
from openai.types.beta.threads.thread_message import ThreadMessage

# First Request
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="tell me the location with map of Los Angles?"
)

In [6]:
available_functions = {
    "travel_map":travel_map ,
} 

In [7]:
from openai.types.beta.threads.run import Run

run: Run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

In [8]:
import time
import json

def show_json(message, obj):
    display(message, json.loads(obj.model_dump_json()))

  # Loop until the run completes or requires action
while True:
    runStatus = client.beta.threads.runs.retrieve(thread_id=thread.id,
                                                  run_id=run.id)
    # Add run steps retrieval here for debuging
    run_steps = client.beta.threads.runs.steps.list(thread_id=thread.id, run_id=run.id)
    # show_json("Run Steps:", run_steps)
    print(runStatus.status ,'.....')

    # This means run is making a function call   
    if runStatus.status == "requires_action":
        print(runStatus.status ,'.....')
        print("Status: ", "requires_action")
        show_json("submit_tool_outputs", runStatus.required_action)
        if runStatus.required_action.submit_tool_outputs and runStatus.required_action.submit_tool_outputs.tool_calls:
            print("toolCalls present:")
            toolCalls = runStatus.required_action.submit_tool_outputs.tool_calls

            tool_outputs = []
            for toolcall in toolCalls:
                function_name = toolcall.function.name
                function_args = json.loads(toolcall.function.arguments)
                
                if function_name in available_functions:
                    
                    
                    function_to_call = available_functions[function_name]
                    print(function_to_call,function_to_call.__name__=="travel_map","================================================================")
                  
                    if function_to_call.__name__ == "travel_map":
                        
                        response = function_to_call(
                        mainLocation=function_args.get("mainLocation"),
                        suggestLocations=function_args.get("suggestLocations")
                        )
                        
                        
                        tool_outputs.append({
                                  "tool_call_id": toolcall.id,
                                  "output": response
                              })
                    
            print(tool_outputs,">>>>>") 
            # Submit tool outputs and update the run
            client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tool_outputs)
      
    elif runStatus.status == "completed":
        # List the messages to get the response
        print("completed...........logic")
        messages: list[ThreadMessage] = client.beta.threads.messages.list(thread_id=thread.id)
        for message in messages.data:
            role_label = "User" if message.role == "user" else "Assistant"
            message_content = message.content[0].text.value
            print(f"{role_label}: {message_content}\n")
        break  # Exit the loop after processing the completed run

    elif run.status == "failed":
      print("Run failed.")
      break

    elif run.status in ["in_progress", "queued"]:
      print(f"Run is {run.status}. Waiting...")
      time.sleep(5)  # Wait for 5 seconds before checking again

    else:
      print(f"Unexpected status: {run.status}")
      break

in_progress .....
Run is queued. Waiting...
requires_action .....
requires_action .....
Status:  requires_action


'submit_tool_outputs'

{'submit_tool_outputs': {'tool_calls': [{'id': 'call_fnB1i0FdWQLzj3hhnxU71eGx',
    'function': {'arguments': '{}', 'name': 'travel_map'},
    'type': 'function'}]},
 'type': 'submit_tool_outputs'}

toolCalls present:
<function travel_map at 0x0000019F350F8900> True ================================================================
None
None


TypeError: 'NoneType' object is not iterable